#Logistic Regression classifier

Classifying data with logit regression.

In [1]:
import numpy as np
import pandas as pd

from scipy.signal import butter, lfilter

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import LeaveOneLabelOut
from sklearn.decomposition import PCA

from glob import glob

from mne.io import RawArray
from mne.channels import read_montage
from mne.epochs import concatenate_epochs
from mne import create_info, find_events, Epochs, concatenate_raws, pick_types
from mne.decoding import CSP

from joblib import Parallel, delayed

import os


 
#############function to read data###########

def prepare_data_train(fname):
    """ read and prepare training data """
    # Read data
    data = pd.read_csv(fname)
    # events file
    events_fname = fname.replace('_data','_events')
    # read event file
    labels= pd.read_csv(events_fname)
    clean=data.drop(['id' ], axis=1)#remove id
    labels=labels.drop(['id' ], axis=1)#remove id
    return  clean,labels

def prepare_data_test(fname):
    """ read and prepare test data """
    # Read data
    data = pd.read_csv(fname)
    return data

scaler= StandardScaler()
nfilters = 4
csp = CSP(n_components=nfilters, reg='lws')
def data_preprocess_train(X, events=[], sca=None, cpast=None, bpass=None, 
                          lpass=None, pca=None, csp_filt=None):
    if csp_filt:
        fit_CSP(data, events)
        X = (csp.filters_[0:nfilters].dot(X.T)).T
    X = np.array(X)
    if pca:
        X = X.dot(pca.components_.T)
    if bpass:
        X = butter_bandpass_filter(X,bpass[0],bpass[1],500)
    if lpass:
        X = butter_lowpass_filter(X,lpass,500)
    if sca:
        X=scaler.fit_transform(X)
    if cpast:
        X = concat_past(X,interval=cpast[0],num_past=cpast[1])
    return X

def data_preprocess_test(X,sca=None, cpast=None, bpass=None, lpass=None, pca=None, csp_filt=None):
    if csp_filt:
        X = (csp.filters_[0:nfilters].dot(X.T)).T
    X = np.array(X)
    if pca:
        X = X.dot(pca.components_.T)
    if lpass:
        X = butter_lowpass_filter(X,lpass,500)
    #f bpass:
        # = butter_bandpass_filter(X,bpass[0],bpass[1],500)
    if sca:
        X=scaler.transform(X)
    if cpast:
        X = concat_past(X,interval=cpast[0],num_past=cpast[1])
    return X

def concat_past(X,interval=20,num_past=5):
    frames = []
    for i in range(num_past):
        X_trunc  = X[i*interval:-(num_past-i)*interval]
        frames.append(X_trunc)
    X_out = np.concatenate(frames,axis=1)
    return X_out


def pd_concat_past(X,interval=20,num_past=5):
    frames = []
    for i in range(num_past):
        X_trunc  = X[i*interval:-(num_past-i)*interval]
        X_trunc = X_trunc.rename(index=lambda x: x-i*interval)
        frames.append(X_trunc)
    X_out = pd.concat(frames,axis=1)
    return X_out

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data,axis=0)
    return y

def butter_lowpass(highcut,fs,order=5):
    nyq = 0.5*fs
    high = highcut/nyq
    b,a = butter(order,high,btype="low")
    return b,a

def butter_lowpass_filter(data,highcut,fs,order=5):
    b, a = butter_lowpass(highcut,fs,order=order)
    y = lfilter(b,a,data,axis=0)
    return y
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
def fit(X,y):
    # Do here you training
    clf = LogisticRegression()
    clf.fit(X,y)
    return clf

def predict(clf,X):
    # do here your prediction
    preds = clf.predict_proba(X)
    return np.atleast_2d(preds[:,clf.classes_==1])

def creat_mne_raw_object(data, events=[]):
    """Create a mne raw instance from csv file"""

    ch_names = list(data.columns)
    
    # read EEG standard montage from mne
    montage = read_montage('standard_1005',ch_names)

    ch_type = ['eeg']*len(ch_names)
    data = np.array(data[ch_names]).T

    if len(events)>0:
        events_names =list(events.columns)
        events_data = np.array(events[events_names]).T     
        # define channel type, the first is EEG, the last 6 are stimulations
        ch_type.extend(['stim']*6)
        ch_names.extend(events_names)
        # concatenate event file and data
        data = np.concatenate((data,events_data))
        
    # create and populate MNE info structure
    info = create_info(ch_names,sfreq=500.0, ch_types=ch_type, montage=montage)
    #info['filename'] = fname
    
    # create raw object 
    raw = RawArray(data,info,verbose=False)
    
    return raw

def fit_CSP(data,events=[]):
    epochs_tot = []
    y = []
    # read and concatenate all the files
    #raw = concatenate_raws([creat_mne_raw_object(fname) for fname in fnames])
    raw = creat_mne_raw_object(data,events=events)
    # pick eeg signal
    picks = pick_types(raw.info,eeg=True)

    events = find_events(raw,stim_channel='HandStart', verbose=False)

    epochs = Epochs(raw, events, {'during' : 1}, 0, 2, proj=False,
                    picks=picks, baseline=None, preload=True,
                    add_eeg_ref=False, verbose=False)

    epochs_tot.append(epochs)
    y.extend([1]*len(epochs))
    
    epochs_rest = Epochs(raw, events, {'before' : 1}, -2, 0, proj=False,
                    picks=picks, baseline=None, preload=True,
                    add_eeg_ref=False, verbose=False)
    
    epochs_rest.times = epochs.times
    
    y.extend([-1]*len(epochs_rest))
    epochs_tot.append(epochs_rest)
        
    epochs = concatenate_epochs(epochs_tot)

    X = epochs.get_data()
    y = np.array(y)
    

    csp.fit(X,y)
    

##downsampling naively like this is not correct, if you do not low pass filter.
##this down sampling here it needed only to keep the script run below 10 minutes.
## please do not downsample or use correct procedure to decimate data without alias
#subsample=100 # training subsample.if you want to downsample the training data
#######columns name for labels#############
cols = ['HandStart','FirstDigitTouch',
        'BothStartLoadPhase','LiftOff',
        'Replace','BothReleased']

In [2]:
def eval_lr(lr,cpast=[],lpass=None,bpass=None,pca=None,sca=None,csp_filt=None):
    fns = glob("../train/subj1_series[2-9]_data.csv")
    scores = np.zeros(len(fns))
    for i, fn in enumerate(fns):
        data, labels = prepare_data_train(fn)
        y_test = labels['FirstDigitTouch']
        if len(cpast)>0:
            y_test = y_test[cpast[0]*cpast[1]:]
        X_test = data_preprocess_test(data,sca=sca,cpast=cpast,bpass=bpass,pca=pca,lpass=lpass,csp_filt=csp_filt)
        pred = lr.predict_proba(X_test)[:,1]
        
        scores[i] = roc_auc_score(y_test,pred)
        
    return np.average(scores)

In [104]:
subsample = 5
cut = 35
cpast = [20,10]
events = ['HandStart','FirstDigitTouch',
            'BothStartLoadPhase','LiftOff',
            'Replace','BothReleased']
subjects = range(1,13)
series = range(1,9)

In [82]:
pred_tot = []
y_tot = []
auc_tot = []
for subject in subjects:
    y_raw= []
    raw = []
    sequence = []

    ################ READ DATA ################################################
    
    for ser in series:
        fname =  '../train/subj%d_series%d_data.csv' % (subject,ser)
        data,labels=prepare_data_train(fname)
        raw.append(data) 
        y_raw.append(labels)
        sequence.extend([ser]*len(data))

    X = pd.concat(raw)
    y = pd.concat(y_raw)
    #transform in numpy array
    #transform train data in numpy array
    X = np.asarray(X.astype(float))
    y = np.asarray(y.astype(float))
    sequence = np.asarray(sequence)


    ################ Train classifiers ########################################
    cv = LeaveOneLabelOut(sequence)
    pred = np.empty((X.shape[0],6))
    
    for train, test in cv:
        X_train = X[train]
        X_test = X[test]
        y_train = y[train][cpast[0]*cpast[1]:]
        #apply preprocessing
        X_train = data_preprocess_train(X_train,labels,lpass=cut,csp_filt=1,sca=1,cpast=cpast)
        X_test=data_preprocess_test(X_test,sca=1,lpass=cut,csp_filt=1,cpast=cpast)
        clfs = Parallel(n_jobs=3)(delayed(fit)(X_train[::subsample,:],y_train[::subsample,i]) for i in range(6))
        preds = Parallel(n_jobs=3)(delayed(predict)(clfs[i],X_test) for i in range(6))
        print(len(test[cpast[0]*cpast[1]:]),len(preds[0]))
        pred[test[cpast[0]*cpast[1]:],:] = np.concatenate(preds,axis=1)
        
    pred_tot.append(pred)
    y_tot.append(y)
    # get AUC
    auc = [roc_auc_score(y[:,i],pred[:,i]) for i in range(6)]     
    auc_tot.append(auc)
    print(auc)

119296 119296
271754 271754
217414 217414
116040 116040
210444 210444
249350 249350
119361 119361
117133 117133
[0.73413156044466588, 0.85468935372119925, 0.84369560297424262, 0.78223508476600656, 0.85561251527460724, 0.84489120509588]
291674 291674
329882 329882
152545 152545
277914 277914
151447 151447
206022 206022
150665 150665


KeyboardInterrupt: 

In [80]:
np.average(auc_tot)

0.66643457002579021

In [51]:
np.average(auc)

0.77000632202767161

In [98]:
idx = []
for i, subject in enumerate(subjects):
    X_test = prepare_data_test('../test/subj%d_series10_data.csv' %(subject))
    idx.append(np.array(X_test['id']))

In [111]:
pred_tot = []
y_tot = []
auc_tot = []
ids_tot = []
idx = []
for i, subject in enumerate(subjects):
    y_raw= []
    raw = []
    raw_test = []
    ################ READ DATA ################################################
    
    for ser in series:
        fname =  '../train/subj%d_series%d_data.csv' % (subject,ser)
        data,labels=prepare_data_train(fname)
        raw.append(data) 
        y_raw.append(labels)   

    for k in range(9,11):
        X_test = prepare_data_test('../test/subj%d_series%d_data.csv' %(subject,k))
        idx.append(np.array(X_test['id']))   
        X_test = X_test.drop(['id'],axis=1)
        raw_test.append(X_test)
    X_test = pd.concat(raw_test)
    X_train = pd.concat(raw)
    y_train = pd.concat(y_raw)
    #transform in numpy array
    #transform train data in numpy array
    X_train = np.asarray(X_train.astype(float))
    y_train = np.asarray(y_train.astype(float))[cpast[0]*cpast[1]:]
    #sequence = np.asarray(sequence)


    ################ Train classifiers ########################################
    #cv = LeaveOneLabelOut(sequence)
    pred = np.empty((X_test.shape[0],6))
    
    #apply preprocessing
    X_train = data_preprocess_train(X_train,labels,lpass=cut,csp_filt=1,sca=1,cpast=cpast)
    X_test=data_preprocess_test(X_test,sca=1,lpass=cut,csp_filt=1,cpast=cpast)
    clfs = Parallel(n_jobs=3)(delayed(fit)(X_train[::subsample,:],y_train[::subsample,i]) for i in range(6))
    preds = Parallel(n_jobs=3)(delayed(predict)(clfs[i],X_test) for i in range(6))
    print(len(test[cpast[0]*cpast[1]:]),len(preds[0]))
    pred[cpast[0]*cpast[1]:,:] = np.concatenate(preds,axis=1)
        
    pred_tot.append(pred)

ids_tot=np.concatenate(idx)
    
submission_file = 'grasp_sub_first.csv'
submission = pd.DataFrame(index=ids_tot,
                          columns=cols,
                          data=np.concatenate(pred_tot))
                   
submission.to_csv(submission_file,index_label='id',float_format='%.3f')

149745 232881
149745 297885
149745 225139
149745 244594
149745 260860
149745 279321
149745 276824
149745 250064
149745 252200
149745 257037
149745 277297
149745 287669


In [113]:
len(ids_tot)

3144171

In [5]:
fn, fn2 = "../train/subj1_series1_data.csv", "../train/subj1_series2_data.csv"
data, labels = prepare_data_train(fn)


In [4]:
fn, fn2 = "../train/subj1_series1_data.csv", "../train/subj1_series2_data.csv"
data, labels = prepare_data_train(fn)

y_train = labels["FirstDigitTouch"]
X_train = data_preprocess_train(data,sca=1)
lr1 = LogisticRegression()
lr1.fit(X_train[::subsample],y_train[::subsample]) 

eval_lr(lr1,sca=1)

/usr/local/lib/python3.4/dist-packages/sklearn/utils/validation.py:498: UserWarning: StandardScaler assumes floating point values as input, got int64
  "got %s" % (estimator, X.dtype))


0.74308979879279602

In [72]:
y_train2 = labels["FirstDigitTouch"][100:]
X_train2 = data_preprocess_train(data,cpast=[20,5],sca=1)
lr2 = LogisticRegression()
lr2.fit(X_train2[::subsample],y_train2[::subsample]) 

eval_lr(lr2,cpast=[20,5],sca=1)

/usr/local/lib/python3.4/dist-packages/sklearn/utils/validation.py:498: UserWarning: StandardScaler assumes floating point values as input, got int64
  "got %s" % (estimator, X.dtype))


0.83923819765432472

In [75]:
band=[0.5,35]
cut = 30

y_train3 = labels["FirstDigitTouch"]
X_train3 = data_preprocess_train(data,lpass=cut,sca=1)
lr3 = LogisticRegression()
lr3.fit(X_train3[::subsample],y_train3[::subsample]) 

eval_lr(lr3,lpass=cut,sca=1)

0.75988739853284293

In [82]:
cut2 = 35

y_train4 = labels["FirstDigitTouch"][100:]
X_train4 = data_preprocess_train(data,lpass=cut2,cpast=[20,5])
lr4 = LogisticRegression()
lr4.fit(X_train4[::subsample],y_train4[::subsample]) 

eval_lr(lr4,lpass=cut2,cpast=[20,5])

0.81979610555588567

In [107]:
cut2 = 35

y_train5 = labels["FirstDigitTouch"]
X_train5 = data_preprocess_train(data,labels,lpass=cut2,csp_filt=1)
lr5 = LogisticRegression()
lr5.fit(X_train5[::subsample],y_train5[::subsample]) 

eval_lr(lr5,lpass=cut2,csp_filt=1)

0.78239142541246032

In [6]:
cut2 = 35
cpast = [19,10]

y_train6 = labels["FirstDigitTouch"][cpast[0]*cpast[1]:]
X_train6 = data_preprocess_train(data,labels,lpass=cut2,cpast=cpast,csp_filt=1)
lr6 = LogisticRegression()
lr6.fit(X_train6[::subsample],y_train6[::subsample]) 

eval_lr(lr6,lpass=cut2,cpast=cpast,csp_filt=1)

0.84497900944935445

In [ ]:
reduced_data = np.zeros((len(data),8))

In [ ]:
reduced_data = data.dot(pca.components_.T)

In [ ]:
reduced_data.shape

In [ ]:
y_train5 = labels["FirstDigitTouch"][100:]
X_train5 = data_preprocess_train(data,bpass=1,cpast=1,pca=pca)
lr5 = LogisticRegression()
lr5.fit(X_train5[::subsample],y_train5[::subsample]) 

In [ ]:
eval_lr(lr5,bpass=1,cpast=1,pca=pca)

In [ ]:
np.where(labels['HandStart']==1)[0][0]

In [39]:
csp = CSP()

In [30]:
X1 = np.array(data[68:1068]).T

In [33]:
X2 = np.array(data[1068:2068]).T

In [34]:
xs = np.array([X1,X2])

In [25]:
ycsp = np.array([1,-1])

In [133]:
1/(25*(1/500))

20.0

In [58]:
data.shape

(119496, 32)

In [60]:
np.array(data)[:,1].shape

(119496,)

In [86]:
list(data.columns[1:])

['Fp2',
 'F7',
 'F3',
 'Fz',
 'F4',
 'F8',
 'FC5',
 'FC1',
 'FC2',
 'FC6',
 'T7',
 'C3',
 'Cz',
 'C4',
 'T8',
 'TP9',
 'CP5',
 'CP1',
 'CP2',
 'CP6',
 'TP10',
 'P7',
 'P3',
 'Pz',
 'P4',
 'P8',
 'PO9',
 'O1',
 'Oz',
 'O2',
 'PO10']

In [87]:
list(data.columns)

['Fp1',
 'Fp2',
 'F7',
 'F3',
 'Fz',
 'F4',
 'F8',
 'FC5',
 'FC1',
 'FC2',
 'FC6',
 'T7',
 'C3',
 'Cz',
 'C4',
 'T8',
 'TP9',
 'CP5',
 'CP1',
 'CP2',
 'CP6',
 'TP10',
 'P7',
 'P3',
 'Pz',
 'P4',
 'P8',
 'PO9',
 'O1',
 'Oz',
 'O2',
 'PO10']

In [109]:
[].empty

AttributeError: 'list' object has no attribute 'empty'

In [44]:
roc_auc_score(labels['HandStart'],np.zeros(len(labels)))

0.5

In [105]:
range(9,10)

range(9, 10)

In [110]:
for k in range(9,11):
    print(k)

9
10
